# import

In [ ]:
import re
import os
import csv
import glob
import base64
import requests
import numpy as np 
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from scipy.stats import zscore

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%capture
!pip install --upgrade --no-cache-dir gdown
!pip install tqdm

# Download and prepare data


because I use google colab to run my code
first of all I download my data from google drive
this data was uploaded manualy on google drive and
are publicly available


In [ ]:
%%capture
!gdown 1_iQKcOhqtCPrta3Cz0pjbO02C2VaQ4wV -O data.rar
!unrar x "/content/data.rar" "/content/main_content/"

# LXML

In [ ]:
! pip install selectolax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 4.7 MB/s eta 0:00:00


In [ ]:
def read_batch(path):

        html64file = pd.read_csv(path, nrows=10)
        htmls_encoded = html64file['html'].to_list()


        return htmls_encoded

html_encoded = read_batch('/content/main_content/isna_all.csv')

In [12]:
import os 
import base64
from selectolax.parser import HTMLParser
from lxml import html 
import pandas as pd 

option = 'selectolax'
parsed_pages = []
for encoded_html in html_encoded:
    unicode_html = base64.b64decode(encoded_html)
    try:
        if option == 'lxlm': 
            parsed_html = html.fromstring(unicode_html)
            body_text = parsed_html.body.text
    
        elif option == 'selectolax': 
            parsed_html = HTMLParser(unicode_html, use_meta_tags=True)
            body_text = parsed_html.body.text()

        if body_text.startswith('402 '):
            pass
        elif body_text.startswith('403 '):
            pass
        elif body_text.startswith('404 '):
            pass 
        else: 
            parsed_pages.append(parsed_html)
    except:
        print("--------")
print(len(parsed_pages), 'of', len(html_encoded), 
            'were read successfuly!')

10 of 10 were read successfuly!


In [18]:

i = 0
for node in parsed_pages[5].body.traverse(include_text = True):
    text = node.text(deep=False, separator='', strip=True)
    if node.tag in ['h1', 'p', 'div'] and len(text) > 0:
        print('node text: %s' % node.text(deep=False, separator='', strip=True))
        i += 1
        # if node.text(deep=False, separator='', strip=True) == description:
        print(i)
        print("node.parent ", node.parent.tag)
        print('---------------------')
        print('attributes: %s' % node.attributes)
        print('tag: %s' % node.tag)
        # print('parent tag: %s' % node.parent.tag)
    if i > 10:
        break

node text: ذخیره نفت سفید در صورت قطعی گاز کافی است
1
node.parent  div
---------------------
attributes: {'class': 'first-title', 'itemprop': 'headline'}
tag: h1
node text: رئیس شرکت ملی پخش فرآورده‌های نفتی ناحیه اردستان گفت: در صورت قطع گازِ مشترکان خانگی تمهیدات لازم برای ذخیره نفت سفید برای شهرستان اردستان اندیشیده شده است.
2
node.parent  div
---------------------
attributes: {'class': 'summary', 'itemprop': 'description'}
tag: p
node text: حمیدرضا فروزنده در گفت‌وگو با ایسنا، اظهار کرد: با وضعیتی که پیش آمده شرایط بحرانی را در بحث سوخت گاز طبیعی در کشور شاهد هستیم که باعث افزایش مصرف سوخت مایع در کشور شده است.
3
node.parent  div
---------------------
attributes: {'style': 'text-align:justify'}
tag: p
node text: وی افزود: صنایع و نیروگاه‌های برق که در ماه‌های گذشته با گاز طبیعی فعالیت می‌کردند، اکنون با سوخت مایع در حال فعالیت هستند، همچنین محدودیت‌هایی در خدمت رسانی جایگاه‌های سوخت «سی. اِن. جی» به وجود آمده که باعث افزایش مصرف سوخت‌های مایع شده است.
4
node.parent  div
-----------

In [25]:
from typing import *
import numpy as np 

class Features():
    '''
    #FIXME: implement functions to extract feature vector from a parsed page 
    '''


    def __init__(self, html_object, n_negative_sample = 100): 
        self.html_object = html_object
        self.og_desc_text = self.find_og_desc_text(html_object)
        self.n_negative_sample = n_negative_sample
        self.desired_tags = ['h1', 'p', 'div']
        self.tag_index = {'div' : 0, 'p': 1, 'h1':2}
        pass

    def get_feature_vectors(self, ) -> Tuple[np.ndarray, List[np.bool8]]: 
        '''
        return a tuple containing the feature vectors alongside of their correspounding labels
        '''
        features = []
        labels = []
        count = 0

        for leaf in self.html_object.body.traverse(include_text=True): 
            text = leaf.text(deep=False, separator='', strip=True)
            if leaf.tag in self.desired_tags and len(text) > 0:
                count += 1

                features.append(self.get_feature_leaf(leaf))
                labels.append(self.check_label(leaf))
            
            if count >= self.n_negative_sample : 
                break

        return [features, labels]

    def check_label(self, node : object) -> bool:
        if self._is_description_node(node):
            return 1
        else:
            return 0

    def find_og_desc_text(self, page : object) -> str:
        head = page.css_first('meta[property="og:description"]')
        if head is None:
            return None

        og_description = head.attributes.get('content')
        if og_description is None:
            return None
        return og_description


    def _is_description_node(self, node):
        return True if node.text(deep=False, separator='', strip=True) == self.og_desc_text else 0



    def get_feature_leaf(self, node_lxml_object) -> np.ndarray: 
        present_nodes = self._get_present_tags(node_lxml_object)
        lenght = self._extract_text_lengh(node_lxml_object)
        depth = self._extract_depth(node_lxml_object)
        word_count = self._extract_word_count(node_lxml_object)

        return [lenght, depth, word_count]



    def _get_present_tags(self, node) -> np.ndarray:
        '''
        returns the present tags of a specefic node as a fixed-size binary array
        ex: [p, div, h1, ...] -> [1, 0, 2, 0, 0, ...]
        '''

        parent_tag = node.parent.tag
        #TODO
        return []



    def _extract_depth(self, node: object) -> int:
        """Write a function to extract the text_lenght inside the lxml_object
        Args:
            lxml_object: a lxml object for each node, children involved
        Returns:
            None
        """
        return 0



    def _extract_text_lengh(self, node: object) -> int:
        """Write a function to extract the text_lenght inside the lxml_object
        Args:
            lxml_object: Dictionary of line data for the coverage file.
        Returns:
            None
        """
        return len(node.text(deep=False, separator='', strip=True))




    def _extract_word_count(self, node: object) -> int:
        """Write a function to extract the text_lenght inside the lxml_object
        Args:
            node: Dictionary of line data for the coverage file.
        Returns:
            None
        """
        return len(list(node.text(deep=False, separator='', strip=True).split()))


In [26]:
extractor = Features(parsed_pages[5])

In [27]:
extractor.og_desc_text

'رئیس شرکت ملی پخش فرآورده\u200cهای نفتی ناحیه اردستان گفت: در صورت قطع گازِ مشترکان خانگی تمهیدات لازم برای ذخیره نفت سفید برای شهرستان اردستان اندیشیده شده است.'

In [28]:
f = extractor.get_feature_vectors()

In [29]:
f

[[[40, 0, 9],
  [156, 0, 27],
  [173, 0, 35],
  [243, 0, 44],
  [356, 0, 68],
  [201, 0, 37],
  [275, 0, 49],
  [378, 0, 68],
  [241, 0, 48],
  [11, 0, 2],
  [32, 0, 8],
  [39, 0, 9],
  [25, 0, 4]],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

# Training model

## Evaluation Metric

In [ ]:
def confusion_matrix_(test_labels, predicted_labels, version):
    if not os.path.exists('result'):
        os.makedirs('result')

    print("\nconfusion_matrix\n")
    cm = confusion_matrix(test_labels, predicted_labels)
    plt.subplots(figsize=(10, 6))
    sn.heatmap(cm, annot = True, fmt = 'd', cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.savefig(f'confusion_matrix_{version}.png', bbox_inches='tight')

def model_result(y_test,y_pred, version):
    ra_score = roc_auc_score(y_test,y_pred)
    a_score = accuracy_score(y_test,y_pred)
    p_score = precision_score(y_test,y_pred)
    r_score = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    
    print('roc_auc_score : ',ra_score)
    print('accuracy_score: ',a_score)
    print('precision_score: ',p_score)
    print('recall_score: ',r_score)
    print('f1-score :', f1)
    print('confusion_matrix: ',confusion_matrix(y_test,y_pred))
    confusion_matrix_(y_test,y_pred, version)

## ML Classifier

In [ ]:
%%capture
!pip install catboost

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.utils import class_weight

def extra_tree_train(x_train, x_test, y_train, y_test):
    clf = ExtraTreesClassifier(n_estimators=10, criterion='entropy')
    #Train the model using the training sets
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test.fillna(0))
    model_result(y_test,y_pred, version='ExtraTreesClassifier_entropy')



def random_forest_classifier(x_train, x_test, y_train, y_test):
    random_forest_clf = RandomForestClassifier(max_depth=20)
    #Train the model using the training sets
    random_forest_clf.fit(x_train, y_train)
    y_pred = random_forest_clf.predict(x_test.fillna(0))
    model_result(y_test,y_pred, version='RandomForestClassifier')



def catboost_classifier(x_train, x_test, y_train, y_test):
    # initializing the model
    cbc = CatBoostClassifier(iterations=100, max_depth=16)

    # training the model
    cbc.fit(x_train, y_train)
    y_pred = cbc.predict(x_test)
    model_result(y_test,y_pred, version='CatBoostClassifier')


def xgboost_classifier(x_train, x_test, y_train, y_test):
    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train, y_train)
    y_pred = xgb_cl.predict(x_test)
    model_result(y_test,y_pred, version='XGBClassifier')
    print(xgb_cl.feature_importances_)
    return xgb_cl

def SVM_classifier(x_train, x_test, y_train, y_test):
    # define outlier detection model
    weight = class_weight.compute_class_weight\
    (class_weight ='balanced', classes = np.unique(y_train), y = y_train)
    weights = {i : weight[i] for i in range(2)}

    svm_model = SVC(class_weight = weights)
    svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)
    model_result(y_test,y_pred, version='SVMClassifier')